<a href="https://colab.research.google.com/github/27Roger/Data-Science/blob/main/IDS_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.stats import mannwhitneyu, ttest_ind
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
df = pd.read_csv("movieReplicationSet.csv")
df.head()
# df.shape

,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II (1985),Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,When watching a movie I cheer or shout or talk or curse at the screen,When watching a movie I feel like the things on the screen are happening to me,As a movie unfolds I start to have problems keeping track of events that happened earlier,"The emotions on the screen ""rub off"" on me - for instance if something sad is happening I get sad or if something frightening is happening I get scared",When watching a movie I get completely immersed in the alternative reality of the film,Movies change my position on social economic or political issues,When watching movies things get so intense that I have to stop watching,Gender identity (1 = female; 2 = male; 3 = self-described),Are you an only child? (1: Yes; 0: No; -1: Did not respond),Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)
0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,1.0,6.0,2.0,5.0,5.0,5.0,1.0,1.0,0,1
1,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,1.0,6.0,5.0,3.0,2.0,1.0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,1,0
3,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,4.0,...,3.0,1.0,1.0,4.0,5.0,3.0,1.0,1.0,0,1
4,NaN,NaN,3.5,NaN,0.5,NaN,0.5,1.0,NaN,0.0,...,2.0,3.0,2.0,5.0,6.0,4.0,4.0,1.0,1,1


In [ ]:
def get_median_distribution(x_columns, y_columns, label_type):
    x_medians = df[x_columns].median()
    y_medians = df[y_columns].median()
    median_of_x = np.median(x_medians.values)
    median_of_y = np.median(y_medians.values)

    # plt.scatter(x_medians.value_counts().keys(), x_medians.value_counts().values, label=label_type)
    # plt.scatter(y_medians.value_counts().keys(), y_medians.value_counts().values, label="Not "+label_type)
    # plt.legend()

    # print(label_type, ": median :", median_of_x)
    # print(x_medians.value_counts())
    # print("\n\nNot "+label_type, ": median :", median_of_y)
    # print(y_medians.value_counts())

    return x_medians, y_medians

def check_category_difference(movie_name, c1_data, c2_data):
  c1 = c1_data[movie_name].values
  c1 = c1[np.isfinite(c1)]
  c2 = c2_data[movie_name].values
  c2 = c2[np.isfinite(c2)]
  t, p = ttest_ind(c1, c2)
  # U1, p = mannwhitneyu(c1, c2, alternative="two-sided",method="exact")
  return(p)

def check_difference(c1_data, c2_data):
  c1 = c1_data.values
  c1 = c1[np.isfinite(c1)]
  c2 = c2_data.values
  c2 = c2[np.isfinite(c2)]
  t, p = ttest_ind(c1, c2)
  return(p)

## 1. Popularity Effect

In [ ]:
movies = df.iloc[:,:400]
popularity_index = (movies.shape[0] - movies.isnull().sum()).sort_values()
not_popular_movies = list(popularity_index[:200].keys())
popular_movies = list(popularity_index[200:].keys())
x, y = get_median_distribution(popular_movies, not_popular_movies, "Popular")
# U1, p = mannwhitneyu(x, y, alternative="greater",method="exact")
# print(U1, p)
# print("\n\nP value =", p)

## 2. New vs Old - Age Effect

In [ ]:
movie_date = []
for column in movies.columns:
    movie_date.append((column, int(column[-5:-1])))
sorted_movie_date = sorted(movie_date, key=lambda x:x[1])
old_movies = sorted_movie_date[:200]
new_movies = sorted_movie_date[200:]
old_movies_list = [x[0] for x in old_movies]
new_movies_list = [x[0] for x in new_movies]
x, y = get_median_distribution(new_movies_list, old_movies_list, "Newer")
# U1, p = mannwhitneyu(x, y, alternative="two-sided",method="exact")
# print(U1, p)
# print("\n\nP value =", p)

## 3. Shrek - Gender Effect

In [ ]:
gender_column = "Gender identity (1 = female; 2 = male; 3 = self-described)"
male_data =  df[df[gender_column]==2].iloc[:,:400]
female_data =  df[df[gender_column]==1].iloc[:,:400]
# self_described_data = df[df[gender_column]==3].iloc[:,:400]

p = check_category_difference("Shrek (2001)", male_data, female_data)
print("P value =", p)

P value = 0.27087511813734183


## 4. Proportion of Movies - Gender Effect

In [ ]:
columns = movies.columns
count = 0
for column in columns:
  p = check_category_difference(column, male_data, female_data)
  if p<=0.005:
    # print(column, p)
    count+=1
print(count, "/",len(columns))
print("Propotion :", count/ len(columns))

46 / 400
Propotion : 0.115


## 5. Lion King - Only Child Effect 

In [ ]:
only_child_columns = "Are you an only child? (1: Yes; 0: No; -1: Did not respond)"
only_child_data =  df[df[only_child_columns]==1].iloc[:,:400]
sibling_data =  df[df[only_child_columns]==0].iloc[:,:400]
p = check_category_difference("The Lion King (1994)", only_child_data, sibling_data)
print("P value =", p)

P value = 0.04026705526268264


## 6. Proportion of Movies - Only Child Effect 

In [ ]:
columns = movies.columns
count = 0
for column in columns:
  p = check_category_difference(column, only_child_data, sibling_data)
  if p<=0.005:
    # print(column, p)
    count+=1
print(count, "/",len(columns))
print("Propotion :", count/ len(columns))

10 / 400
Propotion : 0.025


## 7. The Wolf of Wall Street - Social Watching Effect 

In [ ]:
social_watching_columns = "Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)"
alone_data =  df[df[social_watching_columns]==1].iloc[:,:400]
social_data =  df[df[social_watching_columns]==0].iloc[:,:400]
p = check_category_difference("The Wolf of Wall Street (2013)", alone_data, social_data)
print("P value =", p)

P value = 0.11738913665664574


## 8. Proportion of Movies - Social Watching Effect 

In [ ]:
columns = movies.columns
count = 0
for column in columns:
  p = check_category_difference(column, alone_data, social_data)
  if p<=0.005:
    # print(column, p)
    count+=1
print(count, "/",len(columns))
print("Propotion :", count/ len(columns))

6 / 400
Propotion : 0.015


## 9. Home Alone VS Finding Nemo

In [ ]:
home_alone = df["Home Alone (1990)"]
nemo = df["Finding Nemo (2003)"]
p = check_difference(home_alone, nemo)
print("P value =", p)

P value = 6.405049625567131e-11


## 10. Consistency of Franchises 

In [ ]:
franchises = ["Star Wars", "Harry Potter", "The Matrix", "Indiana Jones", "Jurassic Park", "Pirates of the Caribbean", "Toy Story", "Batman"]
franchise_groups = {}
for franchise in franchises:
  franchise_groups[franchise] = []
  print("\n"+franchise)
  for c in df.columns:
    if franchise in c:
      franchise_groups[franchise].append(c)
      print(c)



Star Wars
Star Wars: Episode IV - A New Hope (1977)
Star Wars: Episode II - Attack of the Clones (2002)
Star Wars: Episode V - The Empire Strikes Back (1980)
Star Wars: Episode 1 - The Phantom Menace (1999)
Star Wars: Episode VII - The Force Awakens (2015)
Star Wars: Episode VI - The Return of the Jedi (1983)

Harry Potter
Harry Potter and the Sorcerer's Stone (2001)
Harry Potter and the Deathly Hallows: Part 2 (2011)
Harry Potter and the Goblet of Fire (2005)
Harry Potter and the Chamber of Secrets (2002)

The Matrix
The Matrix Revolutions (2003)
The Matrix Reloaded (2003)
The Matrix (1999)

Indiana Jones
Indiana Jones and the Last Crusade (1989)
Indiana Jones and the Temple of Doom (1984)
Indiana Jones and the Raiders of the Lost Ark (1981)
Indiana Jones and the Kingdom of the Crystal Skull (2008)

Jurassic Park
The Lost World: Jurassic Park (1997)
Jurassic Park III (2001)
Jurassic Park (1993)

Pirates of the Caribbean
Pirates of the Caribbean: Dead Man's Chest (2006)
Pirates of the

In [ ]:
for k,v in franchise_groups.items():
  print(k,len(v))

Star Wars 6
Harry Potter 4
The Matrix 3
Indiana Jones 4
Jurassic Park 3
Pirates of the Caribbean 3
Toy Story 3
Batman 3


In [ ]:
for c in df.columns:
  if "Nemo" in c:
    print(c)

Finding Nemo (2003)
